In [2]:
%pip install pandas
%pip install numpy
%pip install owlready2
%pip install seaborn
%pip install matplotlib
%pip install cycler
%pip install PyYAML

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Develop

In [126]:
import subprocess
import os
import glob, os
import re
import json
import pandas as pd
import sqlite3
import shutil

# Expect a run directory with multiple .met files and apsimx file with the
# following format
# Expect met file to have the format of '[START YYYY]-[END YYYY]-[tags]-sector-[Sector #].met'
# Expect apsimx file to have the format of '[tags]-sector-[Sector #].apsimx'

apsim_cmd = "/Applications/APSIM2023.5.7200.0.app/Contents/Resources/bin/Models"
run_dir = "../../../ap/run"
os.chdir(run_dir)

In [136]:
def run_model(model):
  return subprocess.Popen([apsim_cmd, model], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True);


In [137]:
# Expect met file to have the format of '[START YYYY]-[END YYYY]-[tags]-sector-[Sector #].met'
# Expect apsimx file to have the format of '[tags]-sector-[Sector #].apsimx'

output_path = "./output/"
shutil.rmtree(output_path, ignore_errors=True)
os.mkdir(output_path)

for sector in range(1, 44):
  suffix = "-sector-" + str(sector)
  models = glob.glob("./input/*" + suffix + ".apsimx")
  weather = glob.glob("./input/*" + suffix + ".met")

  for model in models:
    file = open(model, "r")
    model_src = json.load(file)
    file.close()

    clock = None;
    whether = None;
    for el in model_src["Children"]:
        for el2 in el["Children"]:
            if el2["$type"] == "Models.Clock, Models":
              clock = el2
            if el2["$type"] == "Models.Climate.Weather, Models":
              whether = el2

    if clock == None or whether == None:
      continue

    model_path_ = model.replace("./input/", "")

    for wether_path in weather:
      wether_path_ = wether_path.replace("./input/", "")
      parts = wether_path_.split("-");
      clock["Start"] = parts[0] + "-01-01T00:00:00";
      clock["End"] =  parts[1] + "-12-31T00:00:00";

      whether["FileName"] =  "../" + wether_path
      output_file_name = output_path + model_path_.replace(suffix + ".apsimx","") + "-" + wether_path_.replace(".met", ".apsimx");

      file = open(output_file_name, "w")
      json.dump(model_src, file)
      file.close()


### Invoking APSIMX

In [138]:
regex = 'Saturation of \d*.\d* in layer (\d*) is above acceptable value of (\d*.\d*)'
regex2 = 'Soil water of (\d*.\d*) in layer (\d*) is above saturation of (\d*.\d*)'

# Patches soil sat errors
def try_to_correct_errors(model, output):
    file = open(model, "r")
    model_src = json.load(file)
    file.close()

    sat = []
    for el in model_src["Children"]:
        for el2 in el["Children"]:
            for el3 in el2["Children"]:

                if not "Children" in el3:
                    continue

                for el4 in el3["Children"]:
                    if el4["$type"] == "Models.Soils.Water, Models":
                        initial = el4["InitialValues"]
                        for (_, idx, upperBound) in re.findall(regex2, output):
                            idx = int(idx) - 1
                            initial[idx] = float(upperBound)
                    if el4["$type"] == "Models.Soils.Physical, Models":
                      sat = el4["SAT"]
                      for (idx, upperBound) in re.findall(regex, output):
                          idx = int(idx) - 1
                          sat[idx] = float(upperBound)

    file = open(model, "w")
    json.dump(model_src, file)
    file.close()


In [139]:
models = glob.glob(output_path + "*.apsimx")
models_to_rerun = models


In [140]:
models_to_rerun

['./output/_Soybean-1981-2010-sector-6.apsimx',
 './output/_Soybean-1981-2010-sector-31.apsimx',
 './output/_Maize-1981-2010-sector-32.apsimx',
 './output/_Maize-1981-2010-sector-43.apsimx',
 './output/_Soybean-1981-2010-sector-23.apsimx',
 './output/_Soybean-1981-2010-sector-40.apsimx',
 './output/_Maize-1981-2010-sector-20.apsimx',
 './output/_Soybean-1981-2010-sector-15.apsimx',
 './output/_Maize-1981-2010-sector-16.apsimx',
 './output/_Soybean-1981-2010-sector-19.apsimx',
 './output/_Wheat-1981-2010-sector-14.apsimx',
 './output/_Wheat-1981-2010-sector-2.apsimx',
 './output/_Wheat-1981-2010-sector-18.apsimx',
 './output/_Wheat-1981-2010-sector-30.apsimx',
 './output/_Maize-1981-2010-sector-4.apsimx',
 './output/_Wheat-1981-2010-sector-22.apsimx',
 './output/_Maize-1981-2010-sector-8.apsimx',
 './output/_Wheat-1981-2010-sector-41.apsimx',
 './output/_Wheat-1981-2010-sector-16.apsimx',
 './output/_Wheat-1981-2010-sector-43.apsimx',
 './output/_Wheat-1981-2010-sector-20.apsimx',
 './o

In [142]:
processes = [run_model(model) for model in models_to_rerun]
# for model in models_to_rerun:
#   for path in glob.glob(model.replace(".apsimx", ".db*")):
#     os.remove(path)

models_to_rerun = []
for p in processes:
  p.wait()
  if p.returncode != 0:
    out, err = p.communicate();
    try_to_correct_errors(p.args[1], out)
    models_to_rerun.append(p.args[1])
    print("failed", out)

# Read Outputs

In [114]:
def read_report(sqldb):
    # bog-standard read-write connection
    conn = sqlite3.connect(sqldb)

    report = pd.read_sql_query("SELECT * from report", conn)
    conn.close()

    parts = sqldb.replace(".db", "").replace(output_path, "");
    parts = parts.split("-sector-");
    report["model"] = parts[0]
    report["sector"] = int(parts[1])
    return report


In [115]:
outputs = glob.glob(output_path + "*.db")
reports = [read_report(output) for output in outputs];
reports = pd.concat(reports)
reports.to_csv("./results.csv")